In [20]:
import asyncio as io
import time
from datetime import datetime as dt

In [21]:
def track(function):
    async def wrapper(*args, **kwargs):
        start = dt.now()
        res = await function(*args, **kwargs)
        end = dt.now()
        print("\n\nExecution time: ",(end-start).total_seconds())
        return res
    return wrapper

In [3]:
async def get_data():
    print("GET REQ START---------")
    data = {
        "height": 120,
        "time": "12-45-00",
        "unit": "ft"
    }
    # data get req takes 3 seconds
    await io.sleep(3)
    print("GET REQ END---------")
    return data


async def update_instrument(data):
    for i in range(10):
        print(f"Updating instrument_{i}")
        await io.sleep(0.5)

async def take_readings():
    for i in range(10):
        print(f"Reading instrument_{i}")
        await io.sleep(0.5)


async def upload_readings():
    # uploading reading to the sever
    # it takes 3 seconds
    print("POST REQ START---------")
    await io.sleep(3)
    print("POST REQ STOP---------")
    return True

@track
async def main():
    get_data_cr = io.create_task(get_data())
    take_readings_cr = io.create_task(take_readings())
    d = await get_data_cr
    print("recived data::>>", d)
    
    update_instruments_cr = io.create_task(update_instrument(d))
    await take_readings_cr
    await upload_readings()

# @track
# async def main():
#     await io.gather(count(), count(), count())

if __name__ == "__main__":
    await main()

GET REQ START---------
Reading instrument_0
Reading instrument_1
Reading instrument_2
Reading instrument_3
Reading instrument_4
Reading instrument_5
GET REQ END---------
recived data::>> {'height': 120, 'time': '12-45-00', 'unit': 'ft'}
Updating instrument_0
Reading instrument_6
Updating instrument_1
Reading instrument_7
Updating instrument_2
Reading instrument_8
Updating instrument_3
Reading instrument_9
Updating instrument_4
POST REQ START---------
Updating instrument_5
Updating instrument_6
Updating instrument_7
Updating instrument_8
Updating instrument_9
POST REQ STOP---------


Execution time:  8.012746


In [40]:
async def sleeper(t):
    time.sleep(t)

async def sleep2(t, name="a"):
    for i in range(10):
        print(name,i)
#         await io.sleep(0.3)
        t = io.create_task(sleeper(0.3))
        await t
            
    print("hello")
    
print(1)
t1 = io.create_task(sleep2(4))
t2 = io.create_task(sleep2(4,"b"))
io.gather(sleep2(4,"b"), sleep2(4,"a"))
print("????????")
await t1
await t2


1
????????
a 0
b 0
b 0
a 0
a 1
b 1
b 1
a 1
a 2
b 2
b 2
a 2
a 3
b 3
b 3
a 3
a 4
b 4
b 4
a 4
a 5
b 5
b 5
a 5
a 6
b 6
b 6
a 6
a 7
b 7
b 7
a 7
a 8
b 8
b 8
a 8
a 9
b 9
b 9
a 9
hello
hello
hello
hello


In [56]:
import requests as rq
import json

async def make_r(url):
    return rq.get(url)

async def get_data_list(i=1):
    res = []
    print("S:>> data list")
    for i in range(i):
#         data = rq.get(f"https://reqres.in/api/users?page={i}")
#         time.sleep(0.5)
        t1 = io.create_task(make_r(f"https://reqres.in/api/users?page={i}"))
        data = await t1
        res.append(json.loads(data.text))
    print("D-- data list")
    return res


async def get_data_profile(i=1):
    res = []
    print("S:>> data profile")
    for i in range(i):
        t2 = io.create_task(make_r(f"https://reqres.in/api/unknown/{i}"))
        data = await t2
#         data = rq.get(f"https://reqres.in/api/unknown/{i}")
        res.append(json.loads(data.text))
    print("D:-- data profile")
    return res

@track
async def main():
    t1 = io.create_task(get_data_list(10))
    t2 = io.create_task(get_data_profile(10))
    return await t1 + await t2

print(await main())

S:>> data list
S:>> data profile
D-- data list
D:-- data profile


Execution time:  3.649483
[{'page': 1, 'per_page': 6, 'total': 12, 'total_pages': 2, 'data': [{'id': 1, 'email': 'george.bluth@reqres.in', 'first_name': 'George', 'last_name': 'Bluth', 'avatar': 'https://reqres.in/img/faces/1-image.jpg'}, {'id': 2, 'email': 'janet.weaver@reqres.in', 'first_name': 'Janet', 'last_name': 'Weaver', 'avatar': 'https://reqres.in/img/faces/2-image.jpg'}, {'id': 3, 'email': 'emma.wong@reqres.in', 'first_name': 'Emma', 'last_name': 'Wong', 'avatar': 'https://reqres.in/img/faces/3-image.jpg'}, {'id': 4, 'email': 'eve.holt@reqres.in', 'first_name': 'Eve', 'last_name': 'Holt', 'avatar': 'https://reqres.in/img/faces/4-image.jpg'}, {'id': 5, 'email': 'charles.morris@reqres.in', 'first_name': 'Charles', 'last_name': 'Morris', 'avatar': 'https://reqres.in/img/faces/5-image.jpg'}, {'id': 6, 'email': 'tracey.ramos@reqres.in', 'first_name': 'Tracey', 'last_name': 'Ramos', 'avatar': 'https://reqres.in/img/

In [36]:
async def call_endpoint(endpoint, t=2):
    print(f"Calling endpoint {endpoint}...{dt.now()}")
    await io.sleep(t)
    print(f"Response got from endpoing {endpoint}!...{dt.now()}")
    return endpoint + "-" + str(t)

async def main():
    inputs = (("users",5), ("Orders",3), ("Purchase",5))
    return await io.gather(*(call_endpoint(point, time) for point, time in inputs))

await main()

Calling endpoint users...2023-10-12 17:22:51.603508
Calling endpoint Orders...2023-10-12 17:22:51.603779
Calling endpoint Purchase...2023-10-12 17:22:51.603835
Response got from endpoing Orders!...2023-10-12 17:22:54.607172
Response got from endpoing users!...2023-10-12 17:22:56.606620
Response got from endpoing Purchase!...2023-10-12 17:22:56.606787


['users-5', 'Orders-3', 'Purchase-5']

In [43]:
pip install requests

     |████████████████████████████████| 71kB 2.0MB/s eta 0:00:01
     |████████████████████████████████| 133kB 4.8MB/s eta 0:00:01
     |████████████████████████████████| 51kB 20.8MB/s eta 0:00:01
     |████████████████████████████████| 71kB 6.6MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
